# Black-Scholes Validation: annuity-pricing vs financepy

**Purpose**: Cross-validate our Black-Scholes implementation against financepy

**Reference**: ROADMAP_EXTENDED.md Part I

In [ ]:
import numpy as np
import pandas as pd

# Our implementation
from annuity_pricing.options.pricing.black_scholes import (
    black_scholes_call,
    black_scholes_put,
    black_scholes_with_greeks,
)

# Validator
from financepy.utils.date import Date
from financepy.market.curves.discount_curve_flat import DiscountCurveFlat
from financepy.products.equity.equity_vanilla_option import EquityVanillaOption
from financepy.utils.global_types import OptionTypes
from financepy.models.black_scholes import BlackScholes

print("✅ Imports loaded successfully")

## Test Cases

Golden values from Hull (2018) Ch. 15 and standard test scenarios.

| Test Case | S | K | T | r | σ | q | Source |
|-----------|---|---|---|---|---|---|--------|
| Hull 15.6 | 42 | 40 | 0.5 | 0.10 | 0.20 | 0 | Hull textbook |
| ATM | 100 | 100 | 1.0 | 0.05 | 0.20 | 0 | Standard |
| ITM | 100 | 95 | 1.0 | 0.05 | 0.20 | 0 | Standard |
| OTM | 100 | 110 | 0.5 | 0.03 | 0.25 | 0 | Standard |

In [ ]:
# Test case parameters - including Hull 15.6 golden case
test_cases = [
    {"name": "Hull 15.6", "S": 42, "K": 40, "T": 0.5, "r": 0.10, "sigma": 0.20, "q": 0.0, "expected": 4.76},
    {"name": "ATM", "S": 100, "K": 100, "T": 1.0, "r": 0.05, "sigma": 0.20, "q": 0.0, "expected": None},
    {"name": "ITM", "S": 100, "K": 95, "T": 1.0, "r": 0.05, "sigma": 0.20, "q": 0.0, "expected": None},
    {"name": "OTM", "S": 100, "K": 110, "T": 0.5, "r": 0.03, "sigma": 0.25, "q": 0.0, "expected": None},
]

# Tolerance for comparison (financepy uses slightly different day counting)
TOLERANCE = 0.02  # Within 2 cents for option prices
HULL_TOLERANCE = 0.02  # Hull 15.6 textbook tolerance

In [ ]:
# financepy validation function
def price_with_financepy(S, K, T, r, sigma, q=0.0):
    """Price European call using financepy."""
    valuation_date = Date(1, 1, 2024)
    expiry_date = valuation_date.add_years(T)
    
    discount_curve = DiscountCurveFlat(valuation_date, r)
    dividend_curve = DiscountCurveFlat(valuation_date, q)
    
    option = EquityVanillaOption(
        expiry_date, K, OptionTypes.EUROPEAN_CALL
    )
    
    model = BlackScholes(sigma)
    
    return option.value(
        valuation_date, S, discount_curve, dividend_curve, model
    )

# Run validation - now using real Black-Scholes implementation
results = []
for tc in test_cases:
    fp_price = price_with_financepy(
        tc["S"], tc["K"], tc["T"], tc["r"], tc["sigma"], tc.get("q", 0.0)
    )
    
    # Our implementation - NOW WIRED UP
    our_price = black_scholes_call(
        spot=tc["S"],
        strike=tc["K"],
        rate=tc["r"],
        dividend=tc.get("q", 0.0),
        volatility=tc["sigma"],
        time_to_expiry=tc["T"],
    )
    
    diff = abs(fp_price - our_price)
    tolerance = HULL_TOLERANCE if tc["name"] == "Hull 15.6" else TOLERANCE
    passed = diff < tolerance
    
    results.append({
        "Test": tc["name"],
        "financepy": round(fp_price, 4),
        "Ours": round(our_price, 4),
        "Diff": round(diff, 6),
        "Tolerance": tolerance,
        "Pass": "✅" if passed else "❌",
    })

results_df = pd.DataFrame(results)
print("## Price Comparison: Our BS vs financepy\n")
display(results_df)

# Assertions for validation
all_passed = all(r["Pass"] == "✅" for r in results)
print(f"\n{'✅ All price tests PASSED' if all_passed else '❌ Some tests FAILED'}")

## Greeks Validation

Compare our Greeks (delta, gamma, vega, theta, rho) against financepy for an ATM call.

**Reference values** (ATM: S=K=100, r=5%, σ=20%, T=1, q=0%):
| Greek | Expected | Tolerance |
|-------|----------|-----------|
| Delta | ~0.637 | 0.01 |
| Gamma | ~0.019 | 0.001 |
| Vega | ~37.5 (per 100% vol) | 0.5 |
| Theta | ~-6.4 (per year) | 0.5 |

In [ ]:
# Greeks validation - ATM call
S, K, r, q, sigma, T = 100.0, 100.0, 0.05, 0.0, 0.20, 1.0

# Our implementation returns BSResult with all Greeks
bs_result = black_scholes_with_greeks(
    spot=S,
    strike=K,
    rate=r,
    dividend=q,
    volatility=sigma,
    time_to_expiry=T,
    option_type="call",
)

# financepy Greeks
valuation_date = Date(1, 1, 2024)
expiry_date = valuation_date.add_years(T)
discount_curve = DiscountCurveFlat(valuation_date, r)
dividend_curve = DiscountCurveFlat(valuation_date, q)
option = EquityVanillaOption(expiry_date, K, OptionTypes.EUROPEAN_CALL)
model = BlackScholes(sigma)

fp_delta = option.delta(valuation_date, S, discount_curve, dividend_curve, model)
fp_gamma = option.gamma(valuation_date, S, discount_curve, dividend_curve, model)
fp_vega = option.vega(valuation_date, S, discount_curve, dividend_curve, model)
fp_theta = option.theta(valuation_date, S, discount_curve, dividend_curve, model)

# Our vega is per 1% vol change (multiply by 100 for per 100% vol)
# Our theta is per day (multiply by 365 for per year)
greeks_results = [
    {
        "Greek": "Delta",
        "Ours": round(bs_result.delta, 4),
        "financepy": round(fp_delta, 4),
        "Diff": round(abs(bs_result.delta - fp_delta), 6),
        "Tolerance": 0.01,
        "Pass": "✅" if abs(bs_result.delta - fp_delta) < 0.01 else "❌",
    },
    {
        "Greek": "Gamma",
        "Ours": round(bs_result.gamma, 4),
        "financepy": round(fp_gamma, 4),
        "Diff": round(abs(bs_result.gamma - fp_gamma), 6),
        "Tolerance": 0.001,
        "Pass": "✅" if abs(bs_result.gamma - fp_gamma) < 0.001 else "❌",
    },
    {
        "Greek": "Vega",
        "Ours": round(bs_result.vega, 2),
        "financepy": round(fp_vega, 2),
        "Diff": round(abs(bs_result.vega - fp_vega), 4),
        "Tolerance": 0.5,
        "Pass": "✅" if abs(bs_result.vega - fp_vega) < 0.5 else "❌",
    },
    {
        "Greek": "Theta",
        "Ours": round(bs_result.theta, 4),
        "financepy": round(fp_theta, 4),
        "Diff": round(abs(bs_result.theta - fp_theta), 6),
        "Tolerance": 0.01,  # Per day tolerance
        "Pass": "✅" if abs(bs_result.theta - fp_theta) < 0.01 else "❌",
    },
]

greeks_df = pd.DataFrame(greeks_results)
print("## Greeks Comparison: Our BS vs financepy (ATM Call)\n")
display(greeks_df)

# Assertions
greeks_passed = all(r["Pass"] == "✅" for r in greeks_results)
print(f"\n{'✅ All Greeks tests PASSED' if greeks_passed else '❌ Some Greeks tests FAILED'}")

In [ ]:
# ============================================================
# SUMMARY
# ============================================================
print("=" * 60)
print("VALIDATION SUMMARY: Black-Scholes vs financepy")
print("=" * 60)

price_passed = all(r["Pass"] == "✅" for r in results)
greeks_passed = all(r["Pass"] == "✅" for r in greeks_results)

print(f"\n📊 Price Tests:  {'✅ PASSED' if price_passed else '❌ FAILED'} ({sum(1 for r in results if r['Pass'] == '✅')}/{len(results)})")
print(f"📊 Greeks Tests: {'✅ PASSED' if greeks_passed else '❌ FAILED'} ({sum(1 for r in greeks_results if r['Pass'] == '✅')}/{len(greeks_results)})")

all_passed = price_passed and greeks_passed
print(f"\n{'=' * 60}")
print(f"OVERALL: {'✅ ALL TESTS PASSED' if all_passed else '❌ SOME TESTS FAILED'}")
print(f"{'=' * 60}")

# Raise assertion error if any test failed (for automated runs)
assert all_passed, "Validation failed - see details above"